In [ ]:
#@title Clone the llama.cpp repository
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
/content/llama.cpp


In [ ]:
!git clone https://github.com/akx/ggify.git

Cloning into 'ggify'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 67 (delta 34), reused 54 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 12.89 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
#@title Install cmake and build tools
!apt-get update && apt-get install -y build-essential cmake

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,501 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [ ]:
#@title Build llama.cpp
!cmake . && make

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5")
-- OpenMP found
-- Usi

In [ ]:
# Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title loading vicuna-7b-v1.3 model from hugging-face & saving it

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the tokenizer and model
model_name = "lmsys/vicuna-7b-v1.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# Save as GGML format using llama.cpp compatibility
model.save_pretrained("vicuna_original")
tokenizer.save_pretrained("vicuna_original")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('vicuna_original/tokenizer_config.json',
 'vicuna_original/special_tokens_map.json',
 'vicuna_original/tokenizer.model',
 'vicuna_original/added_tokens.json',
 'vicuna_original/tokenizer.json')

In [ ]:
!mkdir quantized_model

In [ ]:
#@title converting hugging-face model to gguf format
!python3 convert_hf_to_gguf.py /content/llama.cpp/vicuna_original --model-name vicuna --outfile /content/llama.cpp/quantized_model/

INFO:hf-to-gguf:Loading model: vicuna_original
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00003.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {4096, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {11008, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {4096, 4096}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape =

In [ ]:
#@title quantizing gguf fp-16 model to fp-4
!/content/llama.cpp/bin/llama-quantize /content/llama.cpp/quantized_model/vicuna-7B-v1.3-F16.gguf /content/llama.cpp/quantized_model/vicuna-7B_Q4_K_M.gguf Q4_K_M

main: build = 4162 (f6d12e7d)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/llama.cpp/quantized_model/vicuna-7B-v1.3-F16.gguf' to '/content/llama.cpp/quantized_model/vicuna-7B_Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 33 key-value pairs and 291 tensors from /content/llama.cpp/quantized_model/vicuna-7B-v1.3-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = vicuna
llama_model_loader: - kv   3:                            general.version str              = v1.3
llama_model_loader: - kv   4:                       general.organization str       

In [ ]:
#@title inferencing of the quantized model
!/content/llama.cpp/bin/llama-cli -m /content/llama.cpp/quantized_model/vicuna-7B_Q4_K_M.gguf -p "I believe the meaning of life is" -n 128

build: 4162 (f6d12e7d) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 33 key-value pairs and 291 tensors from /content/llama.cpp/quantized_model/vicuna-7B_Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = vicuna
llama_model_loader: - kv   3:                            general.version str              = v1.3
llama_model_loader: - kv   4:                       general.organization str              = Lmsys
llama_model_loader: - kv   5:                           general.basename str   

In [ ]:
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Initialize the custom model
base_model_name = "lmsys/vicuna-7b-v1.3"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

In [ ]:
model= AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float16)

config.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
#@title Medusa Implementation

class medusahead(nn.Module):
  def __init__(self, input_dim, output_dim, weights_data):
    super().__init__()
    self.linear1 = nn.Linear(input_dim, input_dim)
    self.linear2 = nn.Linear(input_dim, output_dim)
    self.silu   = nn.SiLU()

    self._initialize_weights_to_zero(self.linear1)
    # Ensure the weights of linear1 are in float16
    self.linear1.weight.data = self.linear1.weight.data.half()
    if self.linear1.bias is not None:
        self.linear1.bias.data = self.linear1.bias.data.half()


    self.linear2.weight.data.copy_(weights_data)
    self.linear2.weight.data = self.linear2.weight.data.half()
    if self.linear2.bias is not None:
        self.linear2.bias.data = self.linear2.bias.data.half()


  def _initialize_weights_to_zero(self, layer):
    if hasattr(layer, "weight") and layer.weight is not None:
      nn.init.constant_(layer.weight, 0.0)  # Set weights to zero
    if hasattr(layer, "bias") and layer.bias is not None:
      nn.init.constant_(layer.bias, 0.0)  # Set biases to zero


  def forward(self, x):
    residual_x = x
    x= self.linear1(x)
    x= self.silu(x)
    x= x + residual_x
    output= self.linear2(x)

    return output

In [ ]:
#@title Model with Medusa

import json

class model_with_medusa(nn.Module):
  def __init__(self, base_model_name):
    super(model_with_medusa, self).__init__()
    self.model= AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float16)
    self.weights_data= None

    if self.model.lm_head.weight is not None:
      self.weights_data= self.model.lm_head.weight.data

    if self.model.lm_head.bias is not None:
      self.bias_data= self.model.lm_head.bias.data


    # Freeze the base model (optional, if you don't want to fine-tune it)
    for param in self.model.parameters():
        param.requires_grad = False

    # Add a new head (custom linear layer)
    hidden_size = self.model.config.hidden_size
    new_head_out_dim = self.model.config.vocab_size

    self.medusa_head1 = medusahead(hidden_size, new_head_out_dim, self.weights_data)
    self.medusa_head2 = medusahead(hidden_size, new_head_out_dim, self.weights_data)
    self.medusa_head3 = medusahead(hidden_size, new_head_out_dim, self.weights_data)
    self.medusa_head4 = medusahead(hidden_size, new_head_out_dim, self.weights_data)
    self.medusa_head5 = medusahead(hidden_size, new_head_out_dim, self.weights_data)

  def forward(self, input_ids, attention_mask=None):
      # Forward pass through the base model
      outputs = self.model(input_ids, attention_mask=attention_mask, output_hidden_states=True)

      # Extract the final hidden states
      hidden_states = outputs.hidden_states[-1]  # Shape: (batch_size, seq_length, hidden_size)
      # print(f'hidden state data type: {hidden_states.dtype}')

      # Compute the output from the new head
      medusa_head1_output = self.medusa_head1(hidden_states)  # Shape: (batch_size, seq_length, new_head_out_dim)
      medusa_head2_output = self.medusa_head2(hidden_states)
      medusa_head3_output = self.medusa_head3(hidden_states)
      medusa_head4_output = self.medusa_head4(hidden_states)
      medusa_head5_output = self.medusa_head5(hidden_states)

      return {
          "logits": outputs.logits,  # Original language modeling logits
          "medusa_head1_output": medusa_head1_output,  # Output from the new head
          "medusa_head2_output": medusa_head2_output,
          "medusa_head3_output": medusa_head3_output,
          "medusa_head4_output": medusa_head4_output,
          "medusa_head5_output": medusa_head5_output
      }

  def save_pretrained(self, save_directory):
      """Save the model to a directory in Hugging Face format."""
      # Save model weights
      torch.save(self.state_dict(), f"{save_directory}/pytorch_model.bin")

      # Save config
      config = self.model.config.to_dict()
      config["custom_heads"] = ["medusa_head1", "medusa_head2", "medusa_head3", "medusa_head4", "medusa_head5"]
      with open(f"{save_directory}/config.json", "w") as f:
          json.dump(config, f)

      print(f"Model saved to {save_directory}")

In [ ]:
model = model_with_medusa(base_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.state_dict()

OrderedDict([('model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5579e-02, -1.3435e-02,  1.2840e-02,  ...,  1.2833e-02,
                        7.8659e-03, -6.3133e-04],
                      [ 1.6689e-03,  1.6994e-03, -1.4938e-02,  ..., -1.1467e-02,
                       -1.2245e-02, -4.2200e-04],
                      ...,
                      [-9.2163e-03,  2.9800e-02, -1.8066e-02,  ..., -1.1650e-02,
                        1.0414e-02,  4.2305e-03],
                      [-1.2108e-02, -1.4572e-02, -5.6343e-03,  ...,  5.9357e-03,
                       -2.2247e-02, -3.9330e-03],
                      [-1.5976e-02,  9.3412e-04, -2.1225e-02,  ...,  1.3420e-02,
                        2.9282e-02, -8.4610e-03]], dtype=torch.float16)),
             ('model.model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0089, -0.0302,  0.0089,  .

In [ ]:
model.medusa_head1.linear1.weight

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16,
       requires_grad=True)

In [ ]:
import os

def prepare_model_for_gguf_conversion(model, output_dir):
    """
    Prepare a custom model for GGUF conversion by restructuring its state dict

    Args:
        model (nn.Module): Custom model to prepare
        output_dir (str): Directory to save prepared model
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get the state dict
    state_dict = model.state_dict()

    # Create a new state dict with standard HuggingFace naming
    new_state_dict = {}

    # Map custom model layers to standard HuggingFace naming
    layer_mapping = {
        'model.embed_tokens.weight': 'model.model.embed_tokens.weight',
        'model.layers': 'model.model.layers',
        'model.norm.weight': 'model.model.norm.weight',
        'model.lm_head.weight': 'model.lm_head.weight'
    }

    # Custom head mapping (optional, modify as needed)
    for i in range(1, 6):
        head_key = f'medusa_head{i}.linear1'
        new_state_dict[f'medusa_head{i}.linear1.weight'] = state_dict[f'{head_key}.weight']

        if f'{head_key}.bias' in state_dict:
            new_state_dict[f'medusa_head{i}.linear1.bias'] = state_dict[f'{head_key}.bias']

        head_key = f'medusa_head{i}.linear2'
        new_state_dict[f'medusa_head{i}.linear2.weight'] = state_dict[f'{head_key}.weight']

        if f'{head_key}.bias' in state_dict:
            new_state_dict[f'medusa_head{i}.linear2.bias'] = state_dict[f'{head_key}.bias']


    # Manually map base model layers
    for key, value in state_dict.items():
        # Look for keys that match standard HuggingFace model structure
        for prefix, replacement in layer_mapping.items():
            if key.startswith(prefix):
                new_key = key.replace(prefix, replacement)
                new_state_dict[new_key] = value

    # Save the restructured model
    pytorch_model_path = os.path.join(output_dir, 'pytorch_model.bin')
    torch.save(new_state_dict, pytorch_model_path)

    # Copy or recreate the config
    config = model.model.config.to_dict()
    config['custom_heads'] = [f'medusa_head{i}' for i in range(1, 6)]

    config_path = os.path.join(output_dir, 'config.json')
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)

    # Copy tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model.model.config._name_or_path)
    tokenizer.save_pretrained(output_dir)

    return output_dir

In [ ]:
!mkdir vicuna-7b-with-medusa
!mkdir vicuna-7b-with-medusa2

In [ ]:
prepare_model_for_gguf_conversion(model, 'vicuna-7b-with-medusa')

'vicuna-7b-with-medusa'

In [ ]:
prepare_model_for_gguf_conversion(model, 'vicuna-7b-with-medusa2')

'vicuna-7b-with-medusa2'

In [ ]:
# Save as GGML format using llama.cpp compatibility
model.save_pretrained("vicuna-7b-with-medusa")
tokenizer.save_pretrained("vicuna-7b-with-medusa")

Model saved to vicuna-7b-with-medusa


('vicuna-7b-with-medusa/tokenizer_config.json',
 'vicuna-7b-with-medusa/special_tokens_map.json',
 'vicuna-7b-with-medusa/tokenizer.model',
 'vicuna-7b-with-medusa/added_tokens.json',
 'vicuna-7b-with-medusa/tokenizer.json')

In [ ]:
!mkdir quantized_models

In [ ]:
!python3 convert_hf_to_gguf.py /content/llama.cpp/vicuna-7b-with-medusa2 --model-name vicuna-7b-v1.3-medusa --outfile /content/llama.cpp/quantized_models/

INFO:hf-to-gguf:Loading model: vicuna-7b-with-medusa2
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
Traceback (most recent call last):
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 4436, in <module>
    main()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 4430, in main
    model_instance.write()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 434, in write
    self.prepare_tensors()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 1657, in prepare_tensors
    super().prepare_tensors()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 298, in prepare_tensors
    for new_name, data_torch in (self.modify_tensors(data_torch, name, bid)):
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 1625, in modify_tensors
    return [(self.map_tensor_name(name), data_torch)]
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 214, in 

In [ ]:
print(model)

model_with_medusa(
  (base_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
    

### Generation from Model with Medusa

In [ ]:
# Example usage
prompt = "The meaning of life is"
input_ids = base_tokenizer(prompt, return_tensors="pt").input_ids
max_length = 50  # Set maximum sequence length

In [ ]:
print(input_ids.size())

torch.Size([1, 6])


In [ ]:
print(input_ids.shape)

torch.Size([1, 6])


In [ ]:
print(input_ids[0][1])

tensor(450)


In [ ]:
with torch.no_grad():
  outputs = model(input_ids)

hidden state data type: torch.float16


In [ ]:
def decoding(tokenizer, output):
  # If logits are present, convert them to token IDs (e.g., argmax for greedy decoding)
  token_ids= []
  for k in output:
    # print(k)
    token_ids.append((torch.argmax(output[k], dim=-1)))

  print(f'length of token ids: {len(token_ids)}')
  print(f'shape of one element in token ids: {(token_ids[0].shape)}')
  for tokens in token_ids:
    print(f'token shape: {tokens.shape}')
    print(tokens)
    print(tokens[0][-1])
    print(tokens[0][-1].shape)

  x= torch.tensor([tokens[0][-1] for tokens in token_ids]).unsqueeze(0)
  print(x)
  print(x.shape)

  # Decode token IDs into text
  decoded_texts= []
  for tkn_id in token_ids:
    decoded_texts.append(tokenizer.batch_decode(tkn_id, skip_special_tokens=True))

  # Print the results
  for idx, text in enumerate(decoded_texts):
      print(f"Output {idx + 1}: {text}")

In [ ]:
decoding(base_tokenizer, outputs)

length of token ids: 6
shape of one element in token ids: torch.Size([1, 6])
token shape: torch.Size([1, 6])
tensor([[29906, 29871,   310,   278,   338,   304]])
tensor(304)
torch.Size([])
token shape: torch.Size([1, 6])
tensor([[29906, 29871,   310,   278,   338,   304]])
tensor(304)
torch.Size([])
token shape: torch.Size([1, 6])
tensor([[29906,   937,   310,   278,   338,   304]])
tensor(304)
torch.Size([])
token shape: torch.Size([1, 6])
tensor([[29906, 29871,   310,   278,   338,   304]])
tensor(304)
torch.Size([])
token shape: torch.Size([1, 6])
tensor([[29906, 29871,   310,   278,   338,   304]])
tensor(304)
torch.Size([])
token shape: torch.Size([1, 6])
tensor([[29906, 29871,   310,   278,   338,   304]])
tensor(304)
torch.Size([])
tensor([[304, 304, 304, 304, 304, 304]])
torch.Size([1, 6])
Output 1: ['2  of the is to']
Output 2: ['2  of the is to']
Output 3: ['2 first of the is to']
Output 4: ['2  of the is to']
Output 5: ['2  of the is to']
Output 6: ['2  of the is to']


In [ ]:
outputs['logits'].shape

torch.Size([1, 6, 32000])

In [ ]:
print(torch.argmax(outputs['logits'][0][5], dim=-1))

tensor(304)


### Speculative Decoding Implementation

In [ ]:
class SpeculativeDecoder:
    def __init__(self, model, tokenizer, max_length=128):
        """
        Initialize the speculative decoder.

        Args:
        - model: The `model_with_medusa` instance.
        - tokenizer: Tokenizer corresponding to the base model.
        - max_length: Maximum length for generated tokens.
        """
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length

    @torch.no_grad()
    def decode(self, prompt, alpha=1):
        """
        Perform speculative decoding.

        Args:
        - prompt: The input text prompt.
        - alpha: The speculative decoding factor (higher values mean more aggressive speculative decoding).

        Returns:
        - Generated text.
        """
        # Tokenize the input prompt
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids

        # Initialize the sequence and attention mask
        sequence = input_ids
        attention_mask = torch.ones_like(sequence)
        token= 0
        medusa_pred_tokens= []

        while token < (self.max_length):


          # 1. Use Medusa heads for faster predictions
          outputs = self.model(input_ids=sequence)

          token_ids= []

          for k in outputs:
            token_ids.append((torch.argmax(outputs[k], dim=-1)))

          print(f'token_ids ka length: {len(token_ids)}')
          print(f'token_ids ke element ka length: {(token_ids[0].shape)}')
          if medusa_pred_tokens is None:
            print('none ke andar')

          if len(medusa_pred_tokens) == 0:
            medusa_pred_tokens.extend(token_ids[1:6])
            print(f'sequence ka shape: {sequence.shape}')
            print(f'medusa pred tokens length: {len(medusa_pred_tokens)}')
            print(f'medusa pred tokens shape: {medusa_pred_tokens[0].shape}')
            print(f'token ids[0] shape: {token_ids[0].shape}')
            print(f'token ids[0][-1] shape: {token_ids[0][-1].shape}')
            print(f'token ids[0][:][-1] shape: {token_ids[0][:][-1].shape}')
            print(f'token ids[0][0][-1].unsqueeze(0).unsqueeze(1) shape: {(token_ids[0][0][-1].unsqueeze(0).unsqueeze(1)).shape}')
            print(f'token ids[0][0]: {token_ids[0][0]}')
            print(f'token ids[0][0][-1]: {token_ids[0][0][-1]}')

            # Append accepted tokens to the sequence
            sequence = torch.cat([sequence, token_ids[0][0][-1].unsqueeze(0).unsqueeze(1)], dim=-1)

            token += 1

          else:
            token += 1

            if medusa_pred_tokens[0][0][-1] == token_ids[0][0][-6]:
              token += 1

              if medusa_pred_tokens[1][0][-1] == token_ids[0][0][-5]:
                token += 1

                if medusa_pred_tokens[2][0][-1] == token_ids[0][0][-4]:
                  token += 1

                  if medusa_pred_tokens[3][0][-1] == token_ids[0][0][-3]:
                    token += 1

                    if medusa_pred_tokens[4][0][-1] == token_ids[0][0][-2]:
                      token += 1
                      medusa_correct= torch.tensor([tokens[0][-1] for tokens in medusa_pred_tokens]).unsqueeze(0)
                      print(f'medusa_correct shape: {medusa_correct.shape}')
                      sequence = torch.cat([sequence , medusa_correct , token_ids[0][0][-1].unsqueeze(0).unsqueeze(1)], dim=-1)
                      print(f'token: {token}')
                      print(sequence)
                      medusa_pred_tokens.extend(token_ids[1:6])


                    else:
                      medusa_correct= torch.tensor([tokens[0][-1] for tokens in medusa_pred_tokens[:4]]).unsqueeze(0)
                      sequence = torch.cat([sequence , medusa_correct , token_ids[0][0][-2].unsqueeze(0).unsqueeze(1)], dim=-1)
                      print(f'token: {token}')
                      print(sequence)
                      medusa_pred_tokens= []


                  else:
                    medusa_correct= torch.tensor([tokens[0][-1] for tokens in medusa_pred_tokens[:3]]).unsqueeze(0)
                    sequence = torch.cat([sequence , medusa_correct , token_ids[0][0][-3].unsqueeze(0).unsqueeze(1)], dim=-1)
                    print(f'token: {token}')
                    print(sequence)
                    medusa_pred_tokens= []


                else:
                  medusa_correct= torch.tensor([tokens[0][-1] for tokens in medusa_pred_tokens[:2]]).unsqueeze(0)
                  sequence = torch.cat([sequence , medusa_correct , token_ids[0][0][-4].unsqueeze(0).unsqueeze(1)], dim=-1)
                  print(f'token: {token}')
                  print(sequence)
                  medusa_pred_tokens= []


              else:
                medusa_correct= torch.tensor([tokens[0][-1] for tokens in medusa_pred_tokens[:1]]).unsqueeze(0)
                sequence = torch.cat([sequence , medusa_correct , token_ids[0][0][-5].unsqueeze(0).unsqueeze(1)], dim=-1)
                print(f'token: {token}')
                print(sequence)
                medusa_pred_tokens= []


            else:
              sequence = torch.cat([sequence , token_ids[0][0][-6].unsqueeze(0).unsqueeze(1)], dim=-1)
              print(f'token: {token}')
              print(sequence)
              medusa_pred_tokens= []


          # Stop if all tokens are accepted or EOS token is generated
          if (sequence == self.tokenizer.eos_token_id).any():
              break

        # Decode the generated token IDs to text
        generated_text = self.tokenizer.decode(sequence[0], skip_special_tokens=True)
        return generated_text

In [ ]:
# Example Usage
base_model_name = "lmsys/vicuna-7b-v1.3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Initialize your model_with_medusa
model = model_with_medusa(base_model_name)

# Initialize the speculative decoder
decoder = SpeculativeDecoder(model=model, tokenizer=base_tokenizer)

# Perform speculative decoding
prompt = "Once upon a time in a galaxy far, far away"
generated_text = decoder.decode(prompt, alpha=1)

print("Generated Text:")
print(generated_text)

hidden state data type: torch.float16
token_ids ka length: 6
token_ids ke element ka length: torch.Size([1, 13])
sequence ka shape: torch.Size([1, 13])
medusa pred tokens length: 5
medusa pred tokens shape: torch.Size([1, 13])
token ids[0] shape: torch.Size([1, 13])
token ids[0][-1] shape: torch.Size([13])
token ids[0][:][-1] shape: torch.Size([13])
token ids[0][0][-1].unsqueeze(0).unsqueeze(1) shape: torch.Size([1, 1])
token ids[0][0]: tensor([29906,   366,   263,   931, 29892,   263,  2319, 29891,  2215, 29892,
         2215,  3448, 29892])
token ids[0][0][-1]: 29892
hidden state data type: torch.float16
token_ids ka length: 6
token_ids ke element ka length: torch.Size([1, 14])
token: 2
tensor([[    1,  9038,  2501,   263,   931,   297,   263, 15400, 29891,  2215,
         29892,  2215,  3448, 29892,  2215]])
hidden state data type: torch.float16
token_ids ka length: 6
token_ids ke element ka length: torch.Size([1, 15])
sequence ka shape: torch.Size([1, 15])
medusa pred tokens length

KeyboardInterrupt: 

In [ ]:

# Generate tokens using speculative decoding
output_ids = speculative_decode(base_model, proxy_model, input_ids, max_length)
output_text = base_tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated text:", output_text)


### Dynamic Batching

In [ ]:
import queue
import time

class DynamicBatching:
    def __init__(self, model_name, max_batch_size=8, timeout=1.0):
        self.max_batch_size = max_batch_size
        self.timeout = timeout
        self.request_queue = queue.Queue()
        self.model_name = model_name

    def add_request(self, request):
        """Add a new request to the queue."""
        self.request_queue.put(request)

    def process_batch(self, model):
        """Collect and process a batch of requests."""
        batch = []
        start_time = time.time()

        # Collect requests until timeout or max batch size
        while len(batch) < self.max_batch_size and (time.time() - start_time) < self.timeout:
            if not self.request_queue.empty():
                batch.append(self.request_queue.get())

        # If a batch has been collected, process it
        if batch:
            # Perform inference (simulated here)
            output = self.infer(batch, model)
            return output
        return None

    def infer(self, batch, model):
        """Simulate inference on a batch."""
        print(f"Processing batch of size {len(batch)}")

        # Initialize the speculative decoder
        decoder = SpeculativeDecoder(model=model, tokenizer=base_tokenizer)

        generated_text = decoder.decode(batch, alpha=1)

        return generated_text  # Return the processed batch

In [ ]:
# testing

# Initialize your model_with_medusa
model = model_with_medusa(base_model_name)

dynamic_batcher = DynamicBatching(max_batch_size=5, timeout=2)

# Simulating incoming requests
for _ in range(10):
    dynamic_batcher.add_request("I love doing")

# Process and get the result
output = dynamic_batcher.process_batch(model)
print(output)

### Appendix

In [ ]:
import torch
import torch.nn as nn

class medusahead(nn.Module):
  def __init__(self, input_dim, output_dim, weights_data):
    super().__init__()
    self.linear1 = nn.Linear(input_dim, input_dim)
    self.linear2 = nn.Linear(input_dim, output_dim)
    self.silu   = nn.SiLU()

    self._initialize_weights_to_zero(self.linear1)
    # Ensure the weights of linear1 are in float16
    self.linear1.weight.data = self.linear1.weight.data.half()
    if self.linear1.bias is not None:
        self.linear1.bias.data = self.linear1.bias.data.half()


    self.linear2.weight.data.copy_(weights_data)
    self.linear2.weight.data = self.linear2.weight.data.half()
    if self.linear2.bias is not None:
        self.linear2.bias.data = self.linear2.bias.data.half()


  def _initialize_weights_to_zero(self, layer):
    if hasattr(layer, "weight") and layer.weight is not None:
      nn.init.constant_(layer.weight, 0.0)  # Set weights to zero
    if hasattr(layer, "bias") and layer.bias is not None:
      nn.init.constant_(layer.bias, 0.0)  # Set biases to zero


  def forward(self, x):
    residual_x = x
    x= self.linear1(x)
    x= self.silu(x)
    x= x + residual_x
    output= self.linear2(x)

    return output

In [ ]:
import torch
import torch.nn as nn

class ParallelOutputHead(nn.Module):
    def __init__(self, original_lm_head, num_parallel_layers, hidden_dim, output_dim, weights_data, base_model_name="lmsys/vicuna-7b-v1.3"):
        super().__init__()
        self.output = original_lm_head  # Keep the original layer
        self.parallel_layers = nn.ModuleList(
            [medusahead(hidden_dim, output_dim, weights_data) for _ in range(num_parallel_layers)]
        )

    def forward(self, x):
        # Pass through the original lm_head
        original_output = self.output(x)

        # Pass through parallel layers
        parallel_outputs = [layer(x) for layer in self.parallel_layers]


        # Final output transformation
        return {
            "logits": original_output.logits,  # Original language modeling logits
            "medusa_output": parallel_outputs,  # Output from the new head
                }


In [ ]:
# Define dimensions and number of parallel layers
hidden_dim   = model.config.hidden_size  # 4096 for LLaMA
output_dim   = model.config.vocab_size  # 32000 for LLaMA
weights_data = model.lm_head.weight.data
num_parallel_layers = 3  # Number of parallel linear layers

In [ ]:
# Create the parallel output head
parallel_lm_head = ParallelOutputHead(
    original_lm_head=model.lm_head,
    num_parallel_layers=num_parallel_layers,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    weights_data=weights_data
)

In [ ]:
# Replace the model's lm_head
model.lm_head = parallel_lm_head

In [ ]:
# Verify the modified model architecture
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      

In [ ]:
# Print all keys (parameters and their names)
print("Keys in the model's parameters:")
for name, param in model.named_parameters():
    print(name)

Keys in the model's parameters:
model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.dow

In [ ]:
model.save_pretrained("./modified_vicuna_parallel")
tokenizer.save_pretrained("./modified_vicuna_parallel")

('./modified_vicuna_parallel/tokenizer_config.json',
 './modified_vicuna_parallel/special_tokens_map.json',
 './modified_vicuna_parallel/tokenizer.model',
 './modified_vicuna_parallel/added_tokens.json',
 './modified_vicuna_parallel/tokenizer.json')

In [ ]:
!python3 convert_hf_to_gguf.py /content/llama.cpp/modified_vicuna_parallel --model-name vicuna-7b-v1.3-medusa --outfile /content/llama.cpp/quantized_models/

INFO:hf-to-gguf:Loading model: modified_vicuna_parallel
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00003.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {4096, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {11008, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {4096, 4096}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16